  Author: Ankit Kariryaa, University of Bremen
  
  Modified by Xuehui Pi and Qiuqi Luo



In [1]:
from tensorflow.keras.models import load_model
import numpy as np               # numerical array manipulation
import pandas as pd
import geopandas as gps
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import time
from collections import defaultdict
from functools import reduce
from PIL import Image
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from shapely.geometry import Point, Polygon
from shapely.geometry import mapping, shape
import fiona
import cv2
from tqdm import tqdm
import rasterio.mask
import affine

from core.UNet import UNet
from core.losses import tversky, accuracy, dice_coef, dice_loss, IoU, recall,F1_score, precision
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.frame_info import FrameInfo
from core.dataset_generator import DataGenerator
from core.visualize import display_images

%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools
import matplotlib.patches as patches
import random
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

import tensorflow as tf
print(tf.__version__)

2024-01-19 08:53:42.308692: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


2.4.1


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2024-01-19 08:53:44.419219: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 08:53:44.421199: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-19 08:53:44.422104: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-01-19 08:53:44.449428: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-01-19 08:53:44.449466: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] 

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-01-19 08:53:46.033388: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-19 08:53:46.033782: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-01-19 08:53:46.033823: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-01-19 08:53:46.033900: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-01-19 08:53:46.033915: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2024-01-19 08:53:46.033929: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2024

In [4]:
# Initialize the data related variables used in the notebook 

# For reading the GSW and annotated images generated in the step - 1

base_dir = r'/home/nkd/hbh'
type_num = 5
image_type = '.tif'
ann_type = '.png'
# NDWI_fn = 'ndwi'
# red_fn = 'red'
# blue_fn = 'blue'
# green_fn = 'green'
# swir_fn = 'swir'
annotation_fn = 'annotation'
image_fn = 'image'
# For testing, images are divided into sequential patches 
patch_generation_stratergy = 'sequential'
patch_size = (576,576,6) ## Height * Width * (Input or Output) channels：[GSW, ANNOTATION]
BATCH_SIZE = 16 # Model is evaluated in batches; See https://keras.io/models/model/

# # When stratergy == sequential
step_size = (576,576)

# input_shape = (512,512,5)
# input_image_channel = [0,1,2,3,4]
# input_label_channel = [5]

# input_shape = (512,512,2)
# input_image_channel = [0,1]
# input_label_channel = [2]

input_shape = (576,576,5)
input_image_channel = [0,1,2,3,4]
input_label_channel = [5]

OPTIMIZER_NAME = 'adaDelta'
OPTIMIZER = adaDelta 
OPTIMIZER=tf.train.experimental.enable_mixed_precision_graph_rewrite(OPTIMIZER)

Instructions for updating:
Use tf.keras.mixed_precision. There is a guide at https://www.tensorflow.org/guide/mixed_precision. Alternatively, `tf.compat.v1.mixed_precision.enable_mixed_precision_graph_rewrite` can be used, but this is not recommended for TF2 code.
  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)


2024-01-19 08:53:46.066278: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-19 08:53:46.066608: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-01-19 08:53:46.066638: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-01-19 08:53:46.066704: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2024-01-19 08:53:46.066716: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2024-01-19 08:53:46.066726: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2024

In [14]:
LOSS = dice_loss
# LOSS=tf.keras.losses.BinaryCrossentropy()
# LOSS_NAME = 'tversky'
LOSS_NAME = 'dice_loss'
# modelToEvaluate = os.path.join(base_dir, r'saved_models\UNet\lakes_20230818-2031_AdaDelta_dice_loss_b5_012345_512.h5')
# modelToEvaluate = os.path.join(base_dir, r'saved_models\UNet\lakes_20230819-0235_AdaDelta_dice_loss_b5_normalized_012345_512.h5')
# modelToEvaluate = os.path.join(base_dir, r'saved_models\UNet\lakes_20230819-0232_AdaDelta_dice_loss_b2_normalized_012_512.h5')
modelToEvaluate = os.path.join(base_dir, '5_saved_models/lakes_20240118-1337_AdaDelta_dice_loss_012345_576_epoch60.h5')

In [15]:
#File path for final report 
timestr = time.strftime("%Y%m%d-%H%M")
chf = input_image_channel + input_label_channel
chs = reduce(lambda a,b: a+str(b),   chf, '')
evaluation_report_path = model_path =  os.path.join(base_dir, 'evaluationreport') 
if not os.path.exists(evaluation_report_path):
    os.makedirs(evaluation_report_path)
evaluation_report_filename = os.path.join(evaluation_report_path,'evaluation_per_pixel{}_{}.csv'.format(timestr,chs))
print(evaluation_report_filename) 

/home/nkd/hbh/evaluationreport/evaluation_per_pixel20240119-1107_012345.csv


In [7]:
def readImgs(path_to_write, fn):
    image = rasterio.open(os.path.join(path_to_write, fn))
#     print(fn)
    read_image = image.read()/1000
#     print(read_image[1].max())
    
    comb_img = np.transpose(read_image, axes=(1,2,0))
    annotation_im = Image.open(os.path.join(path_to_write, fn.replace(image_fn,annotation_fn).replace(image_type,ann_type)))
    annotation = np.array(annotation_im)
    f = FrameInfo(comb_img, annotation)
    return f
    
def readFrames(dataType):
    frames=[]
    numList=[]
    print(dataType)
#     for i in range(3,4):
    for i in range(0,type_num):#config.type_num
        path_to_write=os.path.join(base_dir,'patchesReshape/{}/type{}'.format(dataType,i))
        all_files = os.listdir(path_to_write)
        all_files_image = [fn for fn in all_files if fn.startswith(image_fn) and fn.endswith(image_type)]#image.png
        print('type{} image number:{}'.format(i,len(all_files_image)))
        for j, fn in enumerate(all_files_image):
            f= readImgs(path_to_write,fn)
            frames.append(f)
    return frames


In [8]:
frames=readFrames('test')
test_patches = DataGenerator(input_image_channel,patch_size, frames, input_label_channel, augmenter = None).all_sequential_patches(step_size)
print('test patchs number:',len(test_patches[0]))

test
type0 image number:53
type1 image number:48
type2 image number:23
type3 image number:18
type4 image number:3
test patchs number: 2864


In [16]:
#Evaluate model 

def evaluate_model(model_path, evaluation_report_filename):
    print(model_path, evaluation_report_filename)
    model = load_model(model_path, custom_objects={'dice_loss':dice_loss, 'accuracy':accuracy , 'recall':recall, 'precision':precision,'IoU':IoU}, compile=False)
    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy,recall,precision,F1_score,IoU])
    
    print('Evaluating model now!')
    ev = model.evaluate(test_patches[0], test_patches[1],  verbose=1, use_multiprocessing=False)
    report  = dict(zip(model.metrics_names, ev))
    report['model_path'] =  model_path   
    report['test_frame_dir']= base_dir   
    report['total_patch_count']= len(test_patches[0])  
    return report

report = evaluate_model(modelToEvaluate, evaluation_report_filename)

/home/nkd/hbh/5_saved_models/lakes_20240118-1337_AdaDelta_dice_loss_012345_576_epoch60.h5 /home/nkd/hbh/evaluationreport/evaluation_per_pixel20240119-1107_012345.csv
Evaluating model now!


2024-01-19 11:07:34.932483: I tensorflow/core/grappler/optimizers/auto_mixed_precision.cc:1960] Converted 0/414 nodes to float16 precision using 0 cast(s) to float16 (excluding Const and Variable casts)
2024-01-19 11:07:34.970178: I tensorflow/core/grappler/optimizers/auto_mixed_precision.cc:1960] Converted 0/397 nodes to float16 precision using 0 cast(s) to float16 (excluding Const and Variable casts)


90/90 [==============================] - 40s 435ms/step - loss: 0.0741 - dice_loss: 0.0740 - accuracy: 0.9965 - recall: 0.9050 - precision: 0.9553 - F1_score: 0.9260 - IoU: 0.8675


In [17]:
# Generate the final report
print(report)

tdf = pd.DataFrame(report, index=[0])  
print(tdf.columns)
col_beginning = ['model_path','test_frame_dir', 'total_patch_count', 'accuracy', 'recall','precision','IoU']

col_rest = [x for x in tdf.columns.tolist() if x not in col_beginning]
cols = col_beginning + col_rest
tdf = tdf[cols]
tdf.to_csv(evaluation_report_filename)

{'loss': 0.08870972692966461, 'dice_loss': 0.08880023658275604, 'accuracy': 0.9960861802101135, 'recall': 0.9028980731964111, 'precision': 0.9309967160224915, 'F1_score': 0.91123366355896, 'IoU': 0.8468655347824097, 'model_path': '/home/nkd/hbh/5_saved_models/lakes_20240118-1337_AdaDelta_dice_loss_012345_576_epoch60.h5', 'test_frame_dir': '/home/nkd/hbh', 'total_patch_count': 2864}
Index(['loss', 'dice_loss', 'accuracy', 'recall', 'precision', 'F1_score',
       'IoU', 'model_path', 'test_frame_dir', 'total_patch_count'],
      dtype='object')
